In [1]:
import re
import sys
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
ROOT = Path('/home/jun0/Projects/emart24')
sys.path.append(str(ROOT))
print(sys.path)

['/home/jun0/Projects/emart24/emart24_codes/KJH', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python38.zip', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8/lib-dynload', '', '/home/jun0/.local/lib/python3.8/site-packages', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8/site-packages', '/home/jun0/.local/lib/python3.8/site-packages/IPython/extensions', '/home/jun0/.ipython', '/home/jun0/Projects/emart24']


In [5]:
# 카테고리 데이터
category = pd.read_excel(ROOT / 'data/category.xlsx')
category.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']

# 상품 마스터 데이터
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})

# shapes
category.shape, item.shape

((3062, 4), (49299, 46))

In [6]:
category.head()

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"
2,1010103,가공식품>조미료>종합조미료>천일염,천일염,"구운천일염, 꽃소금, 굵은소금, 가는소금, 볶은소금"
3,1010104,가공식품>조미료>종합조미료>가공염,가공염,"정제염, 맛소금, 허브솔트, 깨소금"
4,1010105,가공식품>조미료>종합조미료>설탕,설탕,"슈가파우더, 과당, 각설탕, 황설탕, 백설탕, 흑설탕"


In [7]:
all_category = category['현재분류명']
unique_category = set(all_category)
len(all_category), len(unique_category)

(3062, 2818)

In [5]:
# unique 한 상품명
item_unique_bool = ~item['상품명'].duplicated()
# 분류가 겹치는 아이템
overlap = item_unique_bool & (item['대분류명'].isin(unique_category) | item['중분류명'].isin(unique_category) | item['소분류명'].isin(unique_category))

item[overlap].shape

(6966, 46)

In [6]:
대분류만 = item_unique_bool & (item['대분류명'].isin(unique_category) & ~item['중분류명'].isin(unique_category) & ~item['소분류명'].isin(unique_category))
중분류만 = item_unique_bool & (~item['대분류명'].isin(unique_category) & item['중분류명'].isin(unique_category) & ~item['소분류명'].isin(unique_category))
소분류만 = item_unique_bool & (~item['대분류명'].isin(unique_category) & ~item['중분류명'].isin(unique_category) & item['소분류명'].isin(unique_category))
대중분류 = item_unique_bool & (item['대분류명'].isin(unique_category) & item['중분류명'].isin(unique_category) & ~item['소분류명'].isin(unique_category))
중소분류 = item_unique_bool & (~item['대분류명'].isin(unique_category) & item['중분류명'].isin(unique_category) & item['소분류명'].isin(unique_category))
대소분류 = item_unique_bool & (item['대분류명'].isin(unique_category) & ~item['중분류명'].isin(unique_category) & item['소분류명'].isin(unique_category))
대중소분류 = item_unique_bool & (item['대분류명'].isin(unique_category) & item['중분류명'].isin(unique_category) & item['소분류명'].isin(unique_category))

#비중
print('대분류만:', 대분류만.isin([True]).sum(axis=0), ',', 대분류만.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('중분류만:', 중분류만.isin([True]).sum(axis=0), ',', 중분류만.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('소분류만:', 소분류만.isin([True]).sum(axis=0), ',', 소분류만.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('대중분류:', 대중분류.isin([True]).sum(axis=0), ',', 대중분류.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('중소분류:', 중소분류.isin([True]).sum(axis=0), ',', 중소분류.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('대소분류:', 대소분류.isin([True]).sum(axis=0), ',', 대소분류.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')
print('대중소분류:', 대중소분류.isin([True]).sum(axis=0), ',', 대중소분류.isin([True]).sum(axis=0) * 100 / item[overlap].shape[0], '%')

대분류만: 1908 , 27.39018087855297 %
중분류만: 2789 , 40.03732414585128 %
소분류만: 1867 , 26.801607809359748 %
대중분류: 268 , 3.8472581108240025 %
중소분류: 0 , 0.0 %
대소분류: 134 , 1.9236290554120012 %
대중소분류: 0 , 0.0 %


In [7]:
#겹치는거 확인
print('대분류만:', item[대분류만]['대분류명'].unique())
print('중분류만:', item[중분류만]['중분류명'].unique())
print('소분류만:', item[소분류만]['소분류명'].unique())
print('대중분류:', item[대중분류]['중분류명'].unique())
print('대소분류:', item[대소분류]['소분류명'].unique())

대분류만: ['음료' '조리빵']
중분류만: ['스낵' '와인' '김밥' '맥주' '우산' '소주' '전자담배']
소분류만: ['기타가공식품' '인스턴트커피' '칫솔' '기타' '이어폰' '양말' '기타디지털/가전' '접착용품' '기타냉동식품' '각티슈'
 '시리얼' '계란' '치약' '두유' '두부' '기타면류' '탐폰' '면도기' '면도날' '한과' '식빵' '데오드란트' '위스키'
 '기타유가공품']
대중분류: ['파우치']
대소분류: ['숙취해소음료' '전통음료' '탄산수']


In [8]:
item_cat = item[overlap].copy()
item_cat = item_cat.filter(items=['상품명', '대분류명', '중분류명', '소분류명'])

In [9]:
item_cat['대분류_bool'] = 대분류만
item_cat['중분류_bool'] = 중분류만 | 대중분류
item_cat['소분류_bool'] = 소분류만 | 대소분류

In [10]:
#대분류명 '음료'의 경우 category상으로 완전히 다른 곳에 있음
item_cat['카테고리 배치'] = np.nan

for i in range(item_cat.shape[0]):
    if item_cat['대분류_bool'].iloc[i]:
        item_cat['카테고리 배치'].iloc[i] = item_cat['대분류명'].iloc[i]
    elif item_cat['중분류_bool'].iloc[i]:
        item_cat['카테고리 배치'].iloc[i] = item_cat['중분류명'].iloc[i]
    else:
        item_cat['카테고리 배치'].iloc[i] = item_cat['소분류명'].iloc[i]

item_cat = item_cat.drop(columns=['대분류_bool', '중분류_bool', '소분류_bool'])
item_cat

C:\Users\jhkim\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,상품명,대분류명,중분류명,소분류명,카테고리 배치
31,롯데)볼빅500ml,음료,수입음료,수입생수,음료
103,위)밥한그릇뚝딱가쓰오맛27g,가공식품,대용식,기타가공식품,기타가공식품
107,위)블루마운틴블렌드원두70g,기호식품,커피차,인스턴트커피,인스턴트커피
135,프링글스)양파맛1400,과자/빵,스낵,감자스낵,스낵
136,프링글스)오리지널1400,과자/빵,스낵,감자스낵,스낵
...,...,...,...,...,...
49251,농심)켈로그크랜베리아몬드그래놀라300g,가공식품,대용식,시리얼,시리얼
49284,정식품)라잇미닛코코넛190ml16팩,유제품,우유,두유,두유
49289,정식품)라잇미닛코코넛팩190ml,유제품,우유,두유,두유
49293,롯데)레쓰비그란데아메리카노500ml,음료,커피음료(RTD),커피캔음료(대),음료
